In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

def measure_valve_length(image_path):
    """
    Measure valve length with preprocessing options for better accuracy.
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Could not read the image")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Global variables
    points = []
    measurements = []
    ref_horizontal = None
    ref_vertical = None
    current_img = img.copy()
    preprocessing_state = {
        'brightness': 0,
        'contrast': 1.0,
        'sharpness': 0
    }

    def adjust_image(image, brightness=0, contrast=1.0, sharpness=0):
        """Apply basic image adjustments"""
        # Brightness and contrast
        adjusted = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
        
        # Sharpness
        if sharpness != 0:
            blur = cv2.GaussianBlur(adjusted, (3, 3), 0)
            adjusted = cv2.addWeighted(adjusted, 1.0 + sharpness, blur, -sharpness, 0)
            
        return adjusted

    def on_trackbar_change(*args):
        """Handle trackbar changes"""
        # Create a try-except block to handle potential trackbar errors
        try:
            preprocessing_state['brightness'] = cv2.getTrackbarPos('Brightness', 'Controls') - 100
            preprocessing_state['contrast'] = cv2.getTrackbarPos('Contrast', 'Controls') / 50.0
            preprocessing_state['sharpness'] = cv2.getTrackbarPos('Sharpness', 'Controls') / 100.0
        except cv2.error:
            return

        # Apply adjustments
        nonlocal current_img
        current_img = adjust_image(
            img,
            preprocessing_state['brightness'],
            preprocessing_state['contrast'],
            preprocessing_state['sharpness']
        )
        
        # Redraw all points and lines
        temp_img = current_img.copy()
        for i in range(0, len(points), 2):
            if i+1 < len(points):
                cv2.circle(temp_img, points[i], 3, (255, 0, 0), -1)
                cv2.circle(temp_img, points[i+1], 3, (255, 0, 0), -1)
                cv2.line(temp_img, points[i], points[i+1], (0, 255, 0), 2)
        cv2.imshow('Image', temp_img)

    def on_mouse_click(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            temp_img = current_img.copy()
            
            # Draw all points and lines
            for i in range(0, len(points)-1, 2):
                cv2.circle(temp_img, points[i], 3, (255, 0, 0), -1)
                cv2.circle(temp_img, points[i+1], 3, (255, 0, 0), -1)
                cv2.line(temp_img, points[i], points[i+1], (0, 255, 0), 2)
            
            # Draw the latest point
            cv2.circle(temp_img, points[-1], 3, (255, 0, 0), -1)
            
            # If we have a complete measurement
            if len(points) % 2 == 0:
                pt1, pt2 = points[-2], points[-1]
                length_pixels = np.sqrt((pt2[0] - pt1[0])**2 + (pt2[1] - pt1[1])**2)
                measurements.append(length_pixels)
                
                if len(measurements) == 1:
                    nonlocal ref_horizontal
                    ref_horizontal = length_pixels
                    print(f"\nHorizontal reference set: {length_pixels:.2f} pixels (1 cm)")
                    print("Now set vertical reference (1 cm)")
                elif len(measurements) == 2:
                    nonlocal ref_vertical
                    ref_vertical = length_pixels
                    print(f"\nVertical reference set: {length_pixels:.2f} pixels (1 cm)")
                else:
                    # Calculate actual length using calibration
                    dx = pt2[0] - pt1[0]
                    dy = pt2[1] - pt1[1]
                    
                    # Weight horizontal and vertical components
                    horizontal_component = abs(dx) / ref_horizontal if ref_horizontal != 0 else 0
                    vertical_component = abs(dy) / ref_vertical if ref_vertical != 0 else 0
                    
                    length_cm = np.sqrt(horizontal_component**2 + vertical_component**2)
                    
                    # Display measurement
                    mid_point = ((pt1[0] + pt2[0])//2, (pt1[1] + pt2[1])//2)
                    cv2.putText(temp_img, f'{length_cm:.2f} cm', mid_point, 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                    
                    print(f"\nMeasurement {len(measurements)-2}:")
                    print(f"Length: {length_cm:.2f} cm")
                    print(f"      = {length_cm*10:.2f} mm")
                    print(f"      = {length_cm/2.54:.2f} inches")
            
            cv2.imshow('Image', temp_img)
        
        elif event == cv2.EVENT_MOUSEMOVE and len(points) % 2 == 1:
            # Draw preview line
            temp_img = current_img.copy()
            
            # Redraw all complete measurements
            for i in range(0, len(points)-1, 2):
                cv2.circle(temp_img, points[i], 3, (255, 0, 0), -1)
                cv2.circle(temp_img, points[i+1], 3, (255, 0, 0), -1)
                cv2.line(temp_img, points[i], points[i+1], (0, 255, 0), 2)
            
            # Draw the last point and preview line
            cv2.circle(temp_img, points[-1], 3, (255, 0, 0), -1)
            cv2.line(temp_img, points[-1], (x, y), (0, 255, 0), 1)
            
            if len(measurements) >= 2:
                # Calculate preview measurement
                dx = x - points[-1][0]
                dy = y - points[-1][1]
                horizontal_component = abs(dx) / ref_horizontal if ref_horizontal != 0 else 0
                vertical_component = abs(dy) / ref_vertical if ref_vertical != 0 else 0
                preview_length = np.sqrt(horizontal_component**2 + vertical_component**2)
                
                mid_point = ((points[-1][0] + x)//2, (points[-1][1] + y)//2)
                cv2.putText(temp_img, f'{preview_length:.2f} cm', mid_point,
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)
            
            cv2.imshow('Image', temp_img)

    # Create windows first
    cv2.namedWindow('Image')
    cv2.namedWindow('Controls')
    
    # Create trackbars and ensure they're properly initialized
    cv2.createTrackbar('Brightness', 'Controls', 100, 200, lambda x: on_trackbar_change())
    cv2.createTrackbar('Contrast', 'Controls', 50, 100, lambda x: on_trackbar_change())
    cv2.createTrackbar('Sharpness', 'Controls', 0, 100, lambda x: on_trackbar_change())
    
    # Set mouse callback
    cv2.setMouseCallback('Image', on_mouse_click)
    
    # Initial display
    cv2.imshow('Image', current_img)
    
    print("Instructions:")
    print("1. Use the Controls window to adjust image visibility")
    print("2. Click to set reference points (1 cm) - first horizontal, then vertical")
    print("3. Continue clicking to measure distances")
    print("4. Press 'r' to reset, 'q' to quit")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):
            points.clear()
            measurements.clear()
            ref_horizontal = None
            ref_vertical = None
            cv2.imshow('Image', current_img)
            print("\nMeasurements reset. Start with new reference measurements.")
    
    cv2.destroyAllWindows()
    
    # Save final image
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.figure(figsize=(12, 12))
    plt.imshow(current_img)
    plt.title(f'Final Measurements ({len(measurements)-2} measurements)')
    plt.axis('off')
    plt.savefig(f'measurement_result_{timestamp}.png')
    plt.close()

# Usage
if __name__ == "__main__":
    image_path = 
    measure_valve_length(image_path)